In [2]:
import veloxchem as vlx

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 8.


In [3]:
class MockTask:
    def __init__(self, mol, basis, comm, ostream):

        self.molecule = mol
        self.ao_basis = basis
        self.mpi_comm = comm
        self.ostream = ostream

In [4]:
import adcc
from mpi4py import MPI
import sys
import numpy
numpy.set_printoptions(precision=7, suppress=True)

In [5]:
# Input settings
#molecule_string = """
#    O 0 0 0
#    H 0 0 1.795239827225189
#    H 1.693194615993441 0 -0.599043184453037"""

molecule_string = """
    H 0 0 0
    H 0 0 1.795239827225189
    """

basis_set_label = '6-31G'
scf_settings = {'conv_thresh': 1.0e-6}
method_settings = {} #{'xcfun': 'b3lyp', 'grid_level': 4}
rsp_settings = {'conv_thresh': 1.0e-4, 'nstates': 1}

In [6]:
# Communicator and output stream
comm = MPI.COMM_WORLD
ostream = vlx.OutputStream(sys.stdout)

In [7]:
# Molecule and basis set
molecule = vlx.Molecule.read_str(molecule_string, units='au')
basis = vlx.MolecularBasis.read(molecule, basis_set_label)

ostream.print_block(molecule.get_string())
ostream.print_block(basis.get_string('Atomic Basis', molecule))
ostream.flush()

                                              Molecular Geometry (Angstroms)                                              
                                                                                                                          
                          Atom         Coordinate X          Coordinate Y          Coordinate Z                           
                                                                                                                          
                           H           0.000000000000        0.000000000000        0.000000000000                         
                           H           0.000000000000        0.000000000000        0.950000004704                         
                                                                                                                          
                                              Molecular Basis (Atomic Basis)                                              
                

In [8]:
# SCF
task = MockTask(molecule, basis, comm, ostream)
scfdrv = vlx.ScfRestrictedDriver(comm, ostream)
scfdrv.update_settings(scf_settings, method_settings)
scfdrv.compute(molecule, basis)
scfdrv.task = task

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [10]:
# SCF first-order properties
#scf_prop = vlx.ScfFirstOrderProperties(comm, ostream)
#scf_prop.compute(molecule, basis, scfdrv.scf_tensors)
#scf_prop.print_properties(molecule)

In [11]:
# Linear Response
#lr_eig_drv = vlx.LinearResponseEigenSolver(comm, ostream)
#lr_eig_drv.update_settings(rsp_settings, method_settings)
#lr_results = lr_eig_drv.compute(molecule, basis, scfdrv.scf_tensors)
#lr_eig_drv.ostream.flush()

In [12]:
# TDHF/TDA, i.e. CIS
tda_drv = vlx.TDAExciDriver(comm, ostream)
tda_drv.update_settings(rsp_settings, method_settings)
tda_results = tda_drv.compute(molecule, basis, scfdrv.scf_tensors)
tda_drv.ostream.flush()

                                                                                                                          
                                                     TDA Driver Setup                                                     
                                                                                                                          
                               Number of States                : 1                                                        
                               Max. Number of Iterations       : 150                                                      
                               Convergence Threshold           : 1.0e-04                                                  
                               ERI Screening Scheme            : Cauchy Schwarz + Density                                 
                               ERI Screening Threshold         : 1.0e-15                                                  
                

In [13]:
# Get eigenvalues and eigenvectors

nocc = molecule.number_of_alpha_electrons()
mo = scfdrv.scf_tensors['C'] # MO coefficients
mo_occ = mo[:, :nocc]        # occupied
mo_vir = mo[:, nocc:]        # virtual

nocc = mo_occ.shape[1]
nvir = mo_vir.shape[1]

#eig_vals = lr_results["eigenvalues"]
#eig_vecs = lr_results["eigenvectors"]

#eig_vec = eig_vecs[:, 0].copy() # first eigenvector
#print(eig_vec)

In [14]:
# Convert the first eigenvector to matrix form
#eig_vec_as_mat = vlx.LinearResponseEigenSolver.lrvec2mat(eig_vec, nocc, nocc + nvir)

In [15]:
tda_eig_vals = tda_results["eigenvalues"]
tda_eig_vecs = tda_results["eigenvectors"]
tda_size = tda_eig_vecs[:,0].shape
tda_eig_vec=tda_eig_vecs[:,0].copy()
tda_eig_vec_as_mat = tda_eig_vec.reshape(nocc, nvir) #/ numpy.sqrt(2.0)

In [16]:
print(tda_eig_vec)

[-0.9996748  0.        -0.0255009]


In [17]:
#tda_eig_vec_as_mat = vlx.LinearResponseEigenSolver.lrvec2mat(tda_eig_vec, nocc, nocc + nvir)

In [18]:
print(tda_eig_vec_as_mat)

[[-0.9996748  0.        -0.0255009]]


In [19]:
adc1 = adcc.run_adc(scfdrv, method='adc1', n_singlets=1, conv_tol=1e-4)

Starting adc1 singlet Jacobi-Davidson ...
Niter n_ss  max_residual  time  Ritz values
  1     3    1.3509e-32    5ms  [0.4843494]
=== Converged ===
    Number of matrix applies:    3
    Total solver time:            6.419ms


In [22]:
adc1_eig_vec = adc1.excitation_vector[0]["ph"].to_ndarray()
adc1_eig_vals = adc1.excitation_energy

In [23]:
print(adc1_eig_vec)
print(adc1_eig_vals)
print(adc1.describe())

[[ 0.7068768 -0.         0.0180318  0.         0.         0.       ]
 [ 0.         0.         0.         0.7068768 -0.         0.0180318]]
[0.4843494]
+----------------------------------------------------+
| adc1                          singlet ,  converged |
+----------------------------------------------------+
|  #        excitation energy     osc str    |v1|^2  |
|          (au)           (eV)                       |
|  0     0.4843494      13.17982   0.7768         1  |
+----------------------------------------------------+



In [24]:
print(tda_eig_vecs[:,0])
print(tda_eig_vals)

[-0.9996748  0.        -0.0255009]
[0.4843494]


In [25]:
numpy.linalg.norm(adc1_eig_vec)

1.0

In [26]:
0.9999995/numpy.sqrt(2.0)

0.7071064276331569

In [27]:
0.0009539/numpy.sqrt(2.0)

0.0006745091585738476

In [28]:
adc1_eig_vec

array([[ 0.7068768, -0.       ,  0.0180318,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       ,  0.7068768, -0.       ,
         0.0180318]])

In [29]:
vlx_dm_oo = -numpy.einsum('ia,ja->ij', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_dm_vv = numpy.einsum('ia,ib->ab', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_DM_ovov = -numpy.einsum('ib,ja->iajb', tda_eig_vec_as_mat, tda_eig_vec_as_mat)

In [30]:
adcc_dm_oo = -numpy.einsum('ia,ja->ij', adc1_eig_vec, adc1_eig_vec)
adcc_dm_vv = numpy.einsum('ia,ib->ab', adc1_eig_vec, adc1_eig_vec)
adcc_DM_ovov = -numpy.einsum('ib,ja->iajb', adc1_eig_vec, adc1_eig_vec)

foo = adc1.reference_state.fock("o1o1").to_ndarray()
fvv = adc1.reference_state.fock("v1v1").to_ndarray()
ovov = adc1.reference_state.eri("o1v1o1v1").to_ndarray()

excitation_energy = (numpy.einsum('ij,ij->',adcc_dm_oo,foo)
                     + numpy.einsum('ab,ab->', adcc_dm_vv, fvv)
                     + numpy.einsum('iajb,iajb->', adcc_DM_ovov, ovov)
                    )

In [31]:
print(excitation_energy)

0.48434940135740473


In [115]:
vlx_excitation_energy = (2.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
                        +2.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
                        +1.0*(
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, :nvir])
                        +0.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, nvir:])# zero block
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, nocc:, :nvir])
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, nocc:, nvir:])
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, :nocc, :nvir])# zero block
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, :nocc, nvir:])
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, :nvir])
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, nvir:])
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, :nvir, :nocc, :nvir])#zero block
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, :nvir, :nocc, nvir:])
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, :nvir, nocc:, :nvir])
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, :nocc, :nvir])# zero block
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, :nocc, nvir:])# zero block
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, nocc:, :nvir])# zero block
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, nocc:, nvir:])
                        ))

In [116]:
print(vlx_excitation_energy)

0.10401652281676621


In [37]:
print(ovov.shape)
print(ovov)

(2, 6, 2, 6)
[[[[ 0.3374075 -0.        -0.0581306  0.         0.         0.       ]
   [ 0.         0.         0.         0.         0.         0.       ]]

  [[-0.         0.4120629 -0.         0.         0.         0.       ]
   [ 0.         0.         0.         0.         0.         0.       ]]

  [[-0.0581306 -0.         0.4708726  0.         0.         0.       ]
   [ 0.         0.         0.         0.         0.         0.       ]]

  [[ 0.         0.         0.         0.4393378 -0.        -0.139109 ]
   [-0.1019303  0.         0.0809784  0.         0.         0.       ]]

  [[ 0.         0.         0.        -0.         0.5094985  0.       ]
   [ 0.        -0.0974356 -0.         0.         0.         0.       ]]

  [[ 0.         0.         0.        -0.139109  -0.         0.586764 ]
   [ 0.0809784 -0.        -0.1158914  0.         0.         0.       ]]]


 [[[ 0.         0.         0.        -0.1019303  0.         0.0809784]
   [ 0.4393378 -0.        -0.139109   0.         0

In [105]:
print(vlx_DM_ovov)
print()
print(vlx_DM_ovov.shape)
#print(adcc_DM_ovov)

[[[[-0.9993497  0.        -0.0254926]]

  [[ 0.        -0.         0.       ]]

  [[-0.0254926  0.        -0.0006503]]]]

(1, 3, 1, 3)


In [150]:
# Making the Vlx 2PDM twice the size with the other spin blocks
# and assign the corresponding values to it by comparing to the adcc matrix
# afterwards divide by two to yield the same numbers
vlx_DM_ovov_full = numpy.zeros((2*nocc, 2*nvir, 2*nocc, 2*nvir))
#vlx_DM_ovov_full[:nocc, :nvir, :nocc, :nvir] = vlx_DM_ovov
vlx_DM_ovov_full[0, 0, 0, 0] = vlx_DM_ovov_full[0, 3, 1, 0] = vlx_DM_ovov_full[1,0,0,3] = vlx_DM_ovov_full[1,3,1,3] = vlx_DM_ovov[0,0,0,0]
vlx_DM_ovov_full[0, 0, 0, 2] = vlx_DM_ovov_full[0, 2, 0, 0] = vlx_DM_ovov_full[0,3,1,2] = vlx_DM_ovov_full[0,5,1,0] = vlx_DM_ovov[0,2,0,0]
vlx_DM_ovov_full[1, 0, 0, 5] = vlx_DM_ovov_full[1, 2, 0, 3] = vlx_DM_ovov_full[1,3,1,5] = vlx_DM_ovov_full[1,5,1,3] = vlx_DM_ovov[0,2,0,0]
vlx_DM_ovov_full[0, 2, 0, 2] = vlx_DM_ovov_full[0, 5, 1, 2] = vlx_DM_ovov_full[1,2,0,5] = vlx_DM_ovov_full[1,5,1,5] = vlx_DM_ovov[0,2,0,2]

# matrix elements were twice as large
vlx_DM_ovov_full *= 0.5
# norm of the difference between the two density matrices does not seem to vanish entirely...
print(numpy.linalg.norm(adcc_DM_ovov - vlx_DM_ovov_full))
#print(vlx_DM_ovov_full)
vlx_excitation_energy = (2.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
                        +2.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
                        +4.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov_full, ovov)
                        )
print(vlx_excitation_energy)

2.6113194510212303e-08
0.4952591093896861


In [147]:
print(adcc_DM_ovov.shape)
print()
#print(adcc_DM_ovov)
print()
print(adcc_DM_ovov - vlx_DM_ovov_full) # here everything appears to be zero

(2, 6, 2, 6)


[[[[-0.  0.  0. -0. -0. -0.]
   [-0. -0. -0. -0. -0. -0.]]

  [[ 0. -0.  0. -0. -0. -0.]
   [-0. -0. -0. -0. -0. -0.]]

  [[ 0.  0.  0. -0. -0. -0.]
   [-0. -0. -0. -0. -0. -0.]]

  [[-0. -0. -0. -0. -0. -0.]
   [-0.  0.  0. -0. -0. -0.]]

  [[-0. -0. -0. -0. -0. -0.]
   [ 0. -0.  0. -0. -0. -0.]]

  [[-0. -0. -0. -0. -0. -0.]
   [ 0.  0.  0. -0. -0. -0.]]]


 [[[-0. -0. -0. -0.  0.  0.]
   [-0. -0. -0. -0. -0. -0.]]

  [[-0. -0. -0.  0. -0.  0.]
   [-0. -0. -0. -0. -0. -0.]]

  [[-0. -0. -0.  0.  0.  0.]
   [-0. -0. -0. -0. -0. -0.]]

  [[-0. -0. -0. -0. -0. -0.]
   [-0. -0. -0. -0.  0.  0.]]

  [[-0. -0. -0. -0. -0. -0.]
   [-0. -0. -0.  0. -0.  0.]]

  [[-0. -0. -0. -0. -0. -0.]
   [-0. -0. -0.  0.  0.  0.]]]]
